In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Housing in Brazil 🇧🇷 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree

## Import Data

In [ ]:
df=pd.read_excel("/kaggle/input/brazil-csv/Brazil.xlsx")
df

In [ ]:
df.info()
df.head()

## Drop all NAN Values

In [ ]:
df.dropna(inplace=True)

## Split lat-lon at only columns

In [ ]:
df[["lat","lon"]] = df["lat-lon"].str.split(",",expand=True).astype("float")
df.head()

## split Anagoas for state column

In [ ]:
df["state"]=df["place_with_parent_names"].str.split("|",expand=True)[2]
df.head()

## Transform the "price_usd" column of df1 so that all values are floating-point numbers instead of strings.

In [ ]:
df["price_usd"]=(df["price_usd"].replace("$","").replace(",","").astype(float))
df["price_usd"]

In [ ]:
df.drop(columns=df[["place_with_parent_names","lat-lon"]],inplace=True)

## Import the CSV file brasil-real-estate-2.csv into the DataFrame df2

In [ ]:
df2 = pd.read_excel("/kaggle/input/brazil-2/Brazil-2.xlsx")

In [ ]:
df2.info
df2.head

## Use the `"price_brl"` column to create a new column named `"price_usd"`. (Keep in mind that, when this data was collected in 2015 and 2016, a US dollar cost 3.19 Brazilian reals.)

In [ ]:
df2["price_usd"] = df2["price_brl"] / 3.19
df2

In [ ]:
df2.dropna(inplace=True)
df2=df2.drop("price_brl",axis="columns")

## Concatenate df and df2 to create a new DataFrame named dfl

In [ ]:
dfl = pd.concat([df,df2])
print("df shape:", df.shape)

## Use the describe method to create a DataFrame summary_stats with the summary statistics for the "area_m2" and "price_usd" columns.

In [ ]:
summary_stats = dfl[["area_m2","price_usd"]].describe()
summary_stats

In [ ]:
import matplotlib.pyplot as plt
plt.hist(dfl["price_usd"])
plt.xlabel("Price [USD]")
plt.ylabel("Frequency")
plt.title("Distribution of Home Prices")

## reate a horizontal boxplot of "area_m2". Make sure that the x-axis has the label "Area [sq meters]" and the plot has the title "Distribution of Home Sizes". Use Matplotlib (plt).

In [ ]:
plt.boxplot(dfl["area_m2"])
plt.xlabel("Area [sq meters]")
plt.title("Distribution of Home Sizes")


## Use the groupby method to create a Series named mean_price_by_region that shows the mean home price in each region in Brazil, sorted from smallest to largest.

In [ ]:
mean_price_by_region = dfl.groupby("region")["price_usd"].mean().sort_values()
mean_price_by_region

In [ ]:
mean_price_by_region.plot(
    kind="bar",
    xlabel="Region",
    ylabel="Mean Price [USD]",
    title="Mean Home Price by Region"  
);


## Create a DataFrame df_south that contains all the homes from df that are in the "South" region.

In [ ]:
df_south =  dfl.loc[dfl['region'] == 'South']
df_south

In [ ]:
import pandas as pd
homes_by_state = df_south["state"].value_counts()
homes_by_state

In [ ]:
plt.scatter(x=df_south["area_m2"], y=df_south["price_usd"])
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")
plt.title("Rio Grande do Sul: Price vs. Area");

## Create a dictionary south_states_corr, where the keys are the names of the three states in the "South" region of Brazil, and their associated values are the correlation coefficient between "area_m2" and "price_usd" in that state.

As an example, here's a dictionary with the states and correlation coefficients for the Southeast region. Since you're looking at a different region, the states and coefficients will be different, but the structure of the dictionary will be the same.

In [ ]:
corr1= df_south["area_m2"].corr(df_south["price_usd"])
corr1